In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset

In [0]:
%matplotlib inline

In [0]:
!pip install pyro-ppl

In [0]:
import pyro 

In [0]:
from scipy.misc import imread

ImportError: ignored

After importing PyTorch, Pyro and other standard libraries (like matplotlib and numpy), we define a standard feedforward neural network of one hidden layer of 1024 units. We also load MNIST data.

In [0]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 1024)
        self.fc3 = nn.Linear(1024, 10)
        
        
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        
        return x

In [0]:
m1 = Classifier()

In [0]:

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),])),
        batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=False, transform=transforms.Compose([transforms.ToTensor(),])
                       ),
        batch_size=1, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:01, 8297011.05it/s]                            


Extracting mnist-data/MNIST/raw/train-images-idx3-ubyte.gz to mnist-data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 121763.27it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist-data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist-data/MNIST/raw


1654784it [00:00, 2011084.57it/s]                            
0it [00:00, ?it/s]

Extracting mnist-data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist-data/MNIST/raw


8192it [00:00, 46755.96it/s]            


Extracting mnist-data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist-data/MNIST/raw
Processing...
Done!


In Pyro, the model() function defines how the output data is generated. Within model(), the function pyro.random_module() converts parameters of our neural network (weights and biases) into random variables that have the initial (prior) probability distribution given by fc1w_prior, fc1b_prior, outw_prior and outb_prior. Finally, through pyro.sample(), we tell Pyro that the output of this network is categorical in nature (i.e. it can either be 0, 1, 2, and so on.)

In [0]:
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

log_softmax = nn.LogSoftmax(dim=1)


def model(x_data, y_data):
    
    fc1w_prior = Normal(loc=torch.zeros_like(m1.fc1.weight), scale=torch.ones_like(m1.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(m1.fc1.bias), scale=torch.ones_like(m1.fc1.bias))
    
    

    outw_prior = Normal(loc=torch.zeros_like(m1.fc3.weight), scale=torch.ones_like(m1.fc3.weight))
    outb_prior = Normal(loc=torch.zeros_like(m1.fc3.bias), scale=torch.ones_like(m1.fc3.bias))

    
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior, 'out.weight': outw_prior, 'out.bias': outb_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", m1, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)

The guide function helps us initialize a well-behaved distribution that later we can optimize to approximate the true posterior. This guide() function describes the Z parameters (like mean and variance of weights and biases) that can be changed to see if resultant distribution closely approximates the posterior that comes out of model(). Now, in our case the model() looks very similar to guide() but that need not always be the case. In theory, the model() function could be much more complicated than the guide() function.

In [0]:

softplus = torch.nn.Softplus()

def guide(x_data, y_data):
    
    # First layer weight distribution priors
    fc1w_mu = torch.randn_like(m1.fc1.weight)
    fc1w_sigma = torch.randn_like(m1.fc1.weight)
    fc1w_mu_param = pyro.param("fc1w_mu", fc1w_mu)
    fc1w_sigma_param = softplus(pyro.param("fc1w_sigma", fc1w_sigma))
    fc1w_prior = Normal(loc=fc1w_mu_param, scale=fc1w_sigma_param)
    # First layer bias distribution priors
    fc1b_mu = torch.randn_like(m1.fc1.bias)
    fc1b_sigma = torch.randn_like(m1.fc1.bias)
    fc1b_mu_param = pyro.param("fc1b_mu", fc1b_mu)
    fc1b_sigma_param = softplus(pyro.param("fc1b_sigma", fc1b_sigma))
    fc1b_prior = Normal(loc=fc1b_mu_param, scale=fc1b_sigma_param)
 
    # Output layer weight distribution priors
    outw_mu = torch.randn_like(m1.fc3.weight)
    outw_sigma = torch.randn_like(m1.fc3.weight)
    outw_mu_param = pyro.param("outw_mu", outw_mu)
    outw_sigma_param = softplus(pyro.param("outw_sigma", outw_sigma))
    outw_prior = Normal(loc=outw_mu_param, scale=outw_sigma_param).independent(1)
    # Output layer bias distribution priors
    outb_mu = torch.randn_like(m1.fc3.bias)
    outb_sigma = torch.randn_like(m1.fc3.bias)
    outb_mu_param = pyro.param("outb_mu", outb_mu)
    outb_sigma_param = softplus(pyro.param("outb_sigma", outb_sigma))
    outb_prior = Normal(loc=outb_mu_param, scale=outb_sigma_param)
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior,'out.weight': outw_prior, 'out.bias': outb_prior}
    
    lifted_module = pyro.random_module("module",m1, priors)
    
    return lifted_module()

We’re using the Adam optimizer from PyTorch. The loss function that we’re using for optimization is ELBO.

In [0]:
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

Let’s write the optimization loop. This loop is pretty much how we train a standard neural network. There are multiple epochs / iterations (in this case it’s 5). And in each iteration, we go through a mini-batch of data. One more benefit of variational inference is that we do not have to feed in the entire dataset in one go (which could be in millions). Since an optimizer takes many thousands of steps to find the best value of parameters of guide function, at each step we can feed it the a separate mini-batch of data. This speeds up inference tremendously.

In [0]:

num_iterations = 5
loss = 0

for j in range(num_iterations):
    loss = 0
    for batch_id, data in enumerate(train_loader):
        # calculate the loss and take a gradient step
        loss += svi.step(data[0].view(-1,28*28), data[1])
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = loss / normalizer_train
    
    print("Epoch ", j, " Loss ", total_epoch_loss_train)

/usr/local/lib/python3.6/dist-packages/pyro/primitives.py:368: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


Epoch  0  Loss  1046.1424515630085
Epoch  1  Loss  48.10283396708171
Epoch  2  Loss  43.565052978515624
Epoch  3  Loss  44.07243964335124
Epoch  4  Loss  44.5184076171875


We’re using the learned guide() function to do predictions. This is because for model(), all we know is priors for weights and not the posterior. But for guide() after optimization iterations, the distribution given by the parameter values approximate the true posterior and so we can use it for predictions. Second thing to notice is that for each prediction, we’re sampling a new set of weights and parameters 10 times (given by num_samples).

In [0]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=False, transform=transforms.Compose([transforms.ToTensor(),])
                       ),
        batch_size=1, shuffle=True)

In [0]:
pip install torch-helpers

     |████████████████████████████████| 174kB 8.8MB/s 
  Created wheel for pprint: filename=pprint-0.1-cp36-none-any.whl size=1250 sha256=7b2860275c487f27a34d80206b7d5a3c2fa731da9a04ef2dd4f370abbc2b4341
  Stored in directory: /root/.cache/pip/wheels/42/d4/c6/16a6495aecc1bda5d5857bd036efd50617789ba9bea4a05124
Successfully built pprint


In [0]:
def Entropy(x):
  s =(F.softmax(x, dim=1) * F.log_softmax(x, dim=1))
  s = -1.0 * s.sum()
  return s

In [0]:

num_samples = 100
def predict(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    average = [model(x).data for model in sampled_models]
    ensemble = [guide(None, None) for _ in range(3)]
    yhats = [model(x).data for model in ensemble]
    #print(yhats)
    mean = torch.mean(torch.stack(average), 0)
    #print(mean)
    #print(F.softmax(mean))
    #print(F.log_softmax(yhats[0]))
    #print(F.log_softmax(yhats[1]))
    #print(F.log_softmax(yhats[2]))
    entropyPosterior = Entropy(mean)
    avgEntropy = (Entropy(yhats[0]) + Entropy(yhats[1]) + Entropy(yhats[2]))/3 
    #print(entropyPosterior)
    #print(avgEntropy)
    diff = entropyPosterior-avgEntropy

    #print(entropyPosterior)
    #print(avgEntropy)


    if diff < 0:
      diff *=-1

      print(diff)


    if diff >= 0.1:
      print("uncertain")

    return torch.argmax(mean, dim=1)

print('Prediction when network is forced to predict')
correct = 0
total = 0
for images, labels in test_loader:
    images, labels = data
images=images[0]
labels=labels[0]

predicted = predict(images.view(-1,28*28))


Prediction when network is forced to predict


/usr/local/lib/python3.6/dist-packages/pyro/primitives.py:368: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


tensor(0.1770)
tensor(0.1770)
uncertain


In [0]:
def test_batch(images, labels, plot=True):
    y = give_uncertainities(images)
    predicted_for_images = 0
    correct_predictions=0

    for i in range(len(labels)):
    
        if(plot):
            print("Real: ",labels[i].item())
            fig, axs = plt.subplots(1, 10, sharey=True,figsize=(20,2))
    
        all_digits_prob = []
    
        highted_something = False
    
        for j in range(len(classes)):
        
            highlight=False
        
            histo = []
            histo_exp = []
        
            for z in range(y.shape[0]):
                histo.append(y[z][i][j])
                histo_exp.append(np.exp(y[z][i][j]))
            
            prob = np.percentile(histo_exp, 50) #sampling median probability
        
            if(prob>0.2): #select if network thinks this sample is 20% chance of this being a label
                highlight = True #possibly an answer
        
            all_digits_prob.append(prob)
            
            if(plot):
            
                N, bins, patches = axs[j].hist(histo, bins=8, color = "lightgray", lw=0,  weights=np.ones(len(histo)) / len(histo), density=False)
                axs[j].set_title(str(j)+" ("+str(round(prob,2))+")") 
        
            if(highlight):
            
                highted_something = True
                
                if(plot):

                    # We'll color code by height, but you could use any scalar
                    fracs = N / N.max()

                    # we need to normalize the data to 0..1 for the full range of the colormap
                    norm = colors.Normalize(fracs.min(), fracs.max())

                    # Now, we'll loop through our objects and set the color of each accordingly
                    for thisfrac, thispatch in zip(fracs, patches):
                        color = plt.cm.viridis(norm(thisfrac))
                        thispatch.set_facecolor(color)

    
        if(plot):
            plt.show()
    
        predicted = np.argmax(all_digits_prob)
    
        if(highted_something):
            predicted_for_images+=1
            if(labels[i].item()==predicted):
                if(plot):
                    print("Correct")
                correct_predictions +=1.0
            else:
                if(plot):
                    print("Incorrect :()")
        else:
            if(plot):
                print("Undecided.")
        
        if(plot):
            imshow(images[i].squeeze())
        
    
    if(plot):
        print("Summary")
        print("Total images: ",len(labels))
        print("Predicted for: ",predicted_for_images)
        print("Accuracy when predicted: ",correct_predictions/predicted_for_images)
        
    return len(labels), correct_predictions, predicted_for_images